In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
import statsmodels.api as sm

In [8]:
df = pd.read_csv("homework_1.1.csv", index_col=0, skipinitialspace=True)

In [13]:
x = df.drop(columns="Y")
y = df["Y"]
model = LinearRegression()
model.fit(x, y)
print(model.coef_)

[1.00713766 1.96456859 2.97548854]


In [15]:
model_x1 = LinearRegression()
model_x1.fit(df[["X1"]], df["Y"])
print(f"X1: {model_x1.coef_}")

model_x2 = LinearRegression()
model_x2.fit(df[["X2"]], df["Y"])
print(f"X2: {model_x2.coef_}")

model_x3 = LinearRegression()
model_x3.fit(df[["X3"]], df["Y"])
print(f"X3: {model_x3.coef_}")

X1: [1.8417611]
X2: [4.08361258]
X3: [3.0970412]


In [20]:

X = df[["X1", "X2", "X3"]]
X = sm.add_constant(X)  # add intercept
y = df["Y"]

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 3.543e+04
Date:                Fri, 05 Sep 2025   Prob (F-statistic):               0.00
Time:                        20:57:43   Log-Likelihood:                -727.62
No. Observations:                1000   AIC:                             1463.
Df Residuals:                     996   BIC:                             1483.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0026      0.016      0.166      0.8

In [4]:
df1 = pd.read_csv("homework_1.2.csv", index_col=0)

In [7]:
treated = df1[df1["X"] == 1][["Z"]]
control = df1[df1["X"] == 0][["Z"]]

In [10]:
nn = NearestNeighbors(n_neighbors=1, metric="euclidean")
nn.fit(control)
distances, indices = nn.kneighbors(treated)
max_distance = distances.max()
print(max_distance)

0.2102170871093757


In [11]:

# Split treated and control
treated = df1[df1["X"] == 1]
control = df1[df1["X"] == 0]

# Use Z for matching
nn = NearestNeighbors(n_neighbors=1)
nn.fit(control[["Z"]])

# Find nearest control for each treated
distances, indices = nn.kneighbors(treated[["Z"]])

# Get matched control rows
matched_controls = control.iloc[indices.ravel()]

# Compute effect
effect = treated["Y"].mean() - matched_controls["Y"].mean()
print(f"Effect: {effect:.5f}")


Effect: 0.54336


In [12]:

# Split treated and control
treated = df1[df1["X"] == 1]
control = df1[df1["X"] == 0]

# Use Z for matching
nn = NearestNeighbors(radius=0.2)
nn.fit(control[["Z"]])

# Find all neighbors within radius for each treated unit
neighbors = nn.radius_neighbors(treated[["Z"]], return_distance=False)

# Flatten the list of matches
all_matches = [idx for sublist in neighbors for idx in sublist]

# Count duplicates (all but the first occurrence)
from collections import Counter
counts = Counter(all_matches)
duplicates = sum(c - 1 for c in counts.values() if c > 1)

print(f"Number of duplicates: {duplicates}")


Number of duplicates: 685


In [13]:
treated = df1[df1["X"] == 1]
control = df1[df1["X"] == 0]

# Radius-based neighbors
nn = NearestNeighbors(radius=0.2)
nn.fit(control[["Z"]])

neighbors = nn.radius_neighbors(treated[["Z"]], return_distance=False)

# Compute mean Y for each treated unit's matched controls
matched_means = []
for idx, control_idxs in enumerate(neighbors):
    if len(control_idxs) > 0:
        matched_means.append(control.iloc[control_idxs]["Y"].mean())
    else:
        matched_means.append(np.nan)  # if no match, handle as needed

# Compute effect
effect = treated["Y"].mean() - np.nanmean(matched_means)
print(f"Effect: {effect:.4f}")


Effect: 0.5844
